In [1]:
from openai import OpenAI
import json
import os

GPT_MODEL_4 = "gpt-4-0125-preview"
OPEN_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
model = GPT_MODEL_4

def ask(prompt, client, model, temperature = 0):
    response = None
    
    response = client.chat.completions.create(
      model=model,
      messages=prompt,
      temperature=temperature,
      response_format={ "type": "json_object" },
    )

    return response.choices[0].message.content

In [2]:
def write_json_to_txt_raw(json_data, file_name):
    with open(file_name, 'w') as file:
        for test_case, details in json_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [3]:
def write_json_to_txt(json_data, file_name):
    filtered_data = {k: v for k, v in json_data.items() if v.get("reflect") == "yes"}
    with open(file_name, 'w') as file:
        for test_case, details in filtered_data.items():
            file.write(f"Test Case: {test_case}\n")
            for key, value in details.items():
                if isinstance(value, list):
                    file.write(f"{key}:\n")
                    for item in value:
                        file.write(f"  - {item}\n")
                else:
                    file.write(f"{key}: {value}\n")
            file.write("\n")

In [4]:
SYSTEM_PROMPT_8="""
I want you to act as a software tester.
Your task is to read the test scenario's name and the corresponding use case specification to base on those information for generateing test steps for test cases and their following expected result.
Return the test cases in json format.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "explanation": "Why do you create this test case? How does this test case related to the test scenario inputed?",
  ],
}
If there are more than one test case for this scenario, continue writing other test case in this form.

Rules for generating test steps:
- Describe the test step clearly to make sure each test case is independent, tester do not need to read other information (example: other test case, use case specification) to know how to do that step.
- Avoid references to other test cases or instructions like "do as mentioned."
- If the test case need to be repeated to test with different order, data or case, seperate them to be distinct test cases.
- If the scenario is about testing the displation and there is no flow directly cover that scenario, use only the basic (or main) flow to test it.
- If there are use cases mentioned in extended or included use case, create test case combine use cases, Try to find the connection point of use cases for combination. 
- For test scenarios mentioning navigation in the name, only produce test cases related to the specified navigation method. 
(Example: "Scenario: User navigates to a page by navbar", only produce a test case of user navigates to that page by navbar even though the use case description has many way to navigate to that page)
- For test scenarios not mentioning navigation in the name, do not include any navigation test cases.
- Generate test cases that directly match the scenario name. Choose only one flow to cover the scenario.
- Ensure all actions and objectives match the scenario name.
"""

In [5]:
SELF_REF="""
Given a test scenario and test cases to test that given test scenario.
Mark if test case can test the given test scenario or not through test steps, expected output, objective (although if it test other use case path or flow, if it is not used to test the given test scenario.)
And give explanation why you think the resulted test case reflect the given test scenario or not. 
Your response should keep the format of the inserted test cases.
The JSON format should follow the following structure:
{
  "Test Case 1":[
    "testCaseName": "Clear name of the test case so tester know what to test when they first read",
    "objective": "Verify who doing what action or function in the test case and the summary of the final result of the test case",
    "testSteps": [
      "Step 1: Describe the step.",
      "Step 2: Describe the step.",
      "Step 3: Describe the step."
    ],
    "expectedResult": "You inform the tester what should they see after doing all the steps",
    "reflect": "yes/no",
    "explanation": "explain why you think this test case reflect the given test scenario or not",
  ],
}"""

In [6]:
scenarios = ["Successful Purchase with Default Settings", "Buying Products from a Single Store Scenario",
             "Change of Delivery Location Before Purchase", "Change of Payment Method Before Purchase",
             "Applying Platform Promotion Before Purchase", "User Changes Delivery Location to a New Location with Correct Details",
             "Add new delivery location with location name containing numbers", "Add new delivery location with location name less than 2 letters",
             "Add new delivery location with phone number not in format (+84) xxx xxx xxx",
             "Add new delivery location with phone number containing letters",
             "Add new delivery location with exact address not found on the platform's map",
             "User Applies Shop or Product's Promotion Before Placing Order", "User changes shipping option"]

In [7]:
prompt_uc = """
Buy product

Objective: This use case helps user to buy product.
Actor: Users

Preconditions: User must have logged in to be able to access to buy product. User is in the shopping cart page.

Main flow:
Step 1: User choose what product they want to buy by clicking the tick box at the head of the product's block. 
Step 2: User clicks the buy button at the right bottom of the screen.
Step 3: System redirects user to the payment page. There should be the default deliver location already chosen for the user in the delivery location section, the information section of the product including name, options, quantity, price, which shop sells it, promotion information, the shipping information section, the platform's promotion information section and the payment section with the default payment method.
Step 4: User clicks the order button.
Step 5: System removes the product from the shopping cart and forms a new order of that product.

Alternative flow 1: User choose what store they want to buy from
At step 1 of the basic flow: In case all the products they want to buy come from one store, user choose by clicking the tick box at the head of the store.
Step 2: System automatically ticks all the product comes from that store in the user's shopping cart.
Go back to step 2 in the basic flow and continue with the steps from step 2.

Alternative flow 2: User change delivery location to one of the saved location
At step 4 of the basic flow: User clicks on Change title in the delivery location section.
Step 5: User choose other saved location options.
Step 6: System changes the location information in the delivery location section corresponding to what user have chosen.
Go back to step 4 in the basic flow and continue with the steps from step 4.

Alternative flow 3: User change delivery location to a new location
At step 4 of the basic flow: User clicks on Change title in the delivery location section.
Step 5: User clicks Add new location button.
Step 6: User fills in name, a correct name contains only letter and cannot be less than 2 letters. If user fills in wrong format, a notification will appears under the incorrect field and user cannot add location.
Step 7: User fills in phone number, a correct phone number must be in format (+84) xxx xxx xxx with all the x is number. If user fills in wrong format, a notification will appears under the incorrect field and user cannot add location.
Step 8: User chooses city by clicking one of the cities provided by the platform.
Step 9: User chooses district by clicking one of the districts provided by the platform.
Step 10: User chooses ward by clicking one of the wards provided by the platform.
Step 11: User fills in the exact location's address. A correct address must be an address available on the map provided by platform, if it cannot be found in the map, a notification will appears under the incorrect field and user cannot add location.
Step 12: User choose between two options if the location is their own house or their working place.
Step 13: User clicks on Finish button to save the location.
Step 14: System changes the location information in the delivery location section corresponding to what user have chosen.
Go back to step 4 in the basic flow and continue with the steps from step 4.

Alternative flow 4: User adds shop or product's promotion
At step 4 of the basic flow: User clicks on Choose voucher in the product's information section.
Step 5: System shows a list of available vouchers.
Step 6: User chooses one of the available vouchers by clicking on it.
Step 7: System applies the voucher and recalculate the product's price.
Go back to step 4 in the basic flow and continue with the steps from step 4.

Alternative flow 5: User adds platform's promotion
At step 4 of the basic flow: User clicks on Choose voucher in the platform's promotion information section.
Step 5: System shows a list of available vouchers.
Step 6: User chooses one of the available vouchers by clicking on it.
Step 7: System applies the voucher and recalculate the order's price.
Go back to step 4 in the basic flow and continue with the steps from step 4.

Alternative flow 6: User changes shipping option
At step 4 of the basic flow: User clicks on Change title in the shipping information section.
Step 5: System shows a list of available shipping options.
Step 6: User chooses one of the available shipping options by clicking on it.
Step 7: System changes shipping method and recalculate the order's price.
Go back to step 4 in the basic flow and continue with the steps from step 4.

Alternative flow 7: User changes payment method
At step 4 of the basic flow: User clicks on Change title in the payment method section.
Step 5: System shows a list of available payment methods.
Step 6: User chooses one of the available payment methods by clicking on it.
Step 7: System changes payment method.
Go back to step 4 in the basic flow and continue with the steps from step 4.
"""

In [8]:
for scenario in scenarios:
    i = 1
    while(i<=3):
        try: 
            promptTestCase = [
            { "role": "system", "content": SYSTEM_PROMPT_8},
            { "role": "user", "content": scenario + "\n" + prompt_uc}
            ]
            gpt_response = ask(promptTestCase, client, model)
            json_data = json.loads(gpt_response)
            write_json_to_txt_raw(json_data, scenario + "-Raw" + str(i) + ".txt")

            promptSelfRef = [
                { "role": "system", "content": SELF_REF},
                { "role": "user", "content": scenario + "\n" + gpt_response}
            ]
            final_response = ask(promptSelfRef, client, model)
            json_fin = json.loads(final_response)
            write_json_to_txt(json_fin, scenario + str(i) + ".txt")
            i+=1
        except Exception as e:
            print(f"An error occurred with scenario '{scenario}' iteration {i}: {e}")